In [21]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("arrival")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .getOrCreate())
sc = spark.sparkContext

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from glob import glob
import pandas as pd
import uuid

In [3]:
glob('/home/jovyan/work/data/gps_sit/*.csv')

['/home/jovyan/work/data/gps_sit/sitgps_2022-09-02.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-04.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-09.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-11.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-12.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-01.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-03.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-13.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-07.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-10.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-15.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-08.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-05.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-16.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-06.csv',
 '/home/jovyan/work/data/gps_sit/sitgps_2022-09-14.csv']

In [24]:
gps = (spark.read.format('csv')
              .option("header", True)
             .load("/home/jovyan/work/data/gps_sit/*.csv"))
gps = gps.orderBy(gps.datetime_gps.asc())
gps.printSchema()

root
 |-- device: string (nullable = true)
 |-- datetime_gps: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- speed: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- engine_status: string (nullable = true)



In [25]:
gps.count()

7856453

In [26]:
vehicle = (spark.read.format('csv')
            .option("header", True)
            .load('/home/jovyan/work/data/vehicles.csv'))
vehicle.printSchema()

root
 |-- num: string (nullable = true)
 |-- routes: string (nullable = true)
 |-- chasis: string (nullable = true)
 |-- gps: string (nullable = true)



In [27]:
smart_bus = pd.read_excel('/home/jovyan/work/data/smart_bus.xlsx')
smart_bus['routes'] = smart_bus['สายรถ'].str.split('-', n=1).str.get(0)
smart_bus = smart_bus.drop('No.',axis=1)
sb = spark.createDataFrame(smart_bus)
smart_bus.head(10)

,ชื่อผู้ประกอบการ,ยี่ห้อรถ,ประเภทรถ,สายรถ,ทะเบียนรถ,จังหวัด,หมายเลขตัวถัง,วันที่ติดตั้ง,อีมี่ GPS,routes
0,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-01,16-4754,กรุงเทพมหานคร,PRAT29A3UFR000306,2565-07-04 00:00:00,866728061461934,28
1,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-02,16-4768,กรุงเทพมหานคร,PRAT29A3UFR000308,2565-07-04 00:00:00,866728061462346,28
2,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-03,16-4731,กรุงเทพมหานคร,PRAT29A3UFR000309,2565-07-04 00:00:00,867648045405519,28
3,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-04,16-4728,กรุงเทพมหานคร,PRAT29A3UFR000311,2565-07-04 00:00:00,866728061463203,28
4,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-05,16-4756,กรุงเทพมหานคร,PRAT29A3UFR000313,2565-07-04 00:00:00,866728061462437,28
5,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-06,16-4776,กรุงเทพมหานคร,PRAT29A3UFR000316,2565-07-04 00:00:00,867648045372768,28
6,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-07,16-4733,กรุงเทพมหานคร,PRAT29A3UFR000318,2565-07-04 00:00:00,866728061462288,28
7,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-08,16-4729,กรุงเทพมหานคร,PRAT29A3UFR000319,2565-07-04 00:00:00,867648045411350,28
8,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-09,16-4730,กรุงเทพมหานคร,PRAT29A3UFR000321,2565-07-04 00:00:00,866728061461306,28
9,บริษัท สมาร์ทบัส จำกัด,SUNLONG,รถโดยสาร ม.2 (ข),28-10,16-4755,กรุงเทพมหานคร,PRAT29A3UFR000328,2565-07-04 00:00:00,866728061458435,28


In [28]:
sb.printSchema()

root
 |-- ชื่อผู้ประกอบการ: string (nullable = true)
 |-- ยี่ห้อรถ: string (nullable = true)
 |-- ประเภทรถ: string (nullable = true)
 |-- สายรถ: string (nullable = true)
 |-- ทะเบียนรถ: string (nullable = true)
 |-- จังหวัด: string (nullable = true)
 |-- หมายเลขตัวถัง: string (nullable = true)
 |-- วันที่ติดตั้ง: timestamp (nullable = true)
 |-- อีมี่ GPS: long (nullable = true)
 |-- routes: string (nullable = true)



In [29]:
vehicle.limit(5).toPandas()

,num,routes,chasis,gps
0,16-5840 กทม.,35,MRSBCREM3MZM00001,868998031822683
1,16-5843 กทม.,35,MRSBCREM5MZM00033,864507039673982
2,16-5845 กทม.,35,MRSBCREM8MZM00057,868998030634774
3,16-5844 กทม.,35,MRSBCREM1MZM00062,868998030615351
4,16-5847 กทม.,35,MRSBCREM7MZM00034,868998030627521


In [30]:
v_list = vehicle.toPandas()['gps'].tolist()
sb_list = smart_bus['อีมี่ GPS'].tolist()
gps_list = gps.select('device').distinct().toPandas()['device'].astype(int).tolist()

In [31]:
set(v_list).isdisjoint(set(gps_list))

True

In [32]:
set(sb_list).isdisjoint(set(gps_list))

True

In [33]:
set.intersection(set(v_list),set(gps_list)), set.intersection(set(sb_list),set(gps_list))

(set(), set())

In [34]:
vehicle.select('routes').distinct().show()

+------+
|routes|
+------+
|   133|
|    35|
|     6|
|    56|
|    39|
|    80|
+------+



In [35]:
gps.limit(5).toPandas()

,device,datetime_gps,latitude,longitude,speed,odometer,engine_status
0,864352043372192,2022-09-01 00:00:00.000,13.787150,100.678210,0.00,2107603.0,0
1,864352041913328,2022-09-01 00:00:00.000,13.787181,100.678031,0.00,730057.0,1
2,864352041913344,2022-09-01 00:00:00.000,13.787151,100.678166,0.00,755866.0,0
3,864352043379783,2022-09-01 00:00:01.000,13.787218,100.677975,0.00,752541.0,1
4,864606045195285,2022-09-01 00:00:02.000,13.787255,100.678026,0.00,565847.0,1


In [36]:
gps_join = gps.join(sb,gps.device ==  sb['อีมี่ GPS'],"inner")

In [37]:
gps_join.limit(10).toPandas()

,device,datetime_gps,latitude,longitude,speed,odometer,engine_status,ชื่อผู้ประกอบการ,ยี่ห้อรถ,ประเภทรถ,สายรถ,ทะเบียนรถ,จังหวัด,หมายเลขตัวถัง,วันที่ติดตั้ง,อีมี่ GPS,routes


In [38]:
gps_join.count()

0